## Lambda Test Event

In [1]:
{
  "s3-dataset-uri": "udacity-ml-workflow/Lesson 3, Exercise 1/reviews_Patio_Lawn_and_Garden_5.json.zip"
}

{'s3-dataset-uri': 'udacity-ml-workflow/Lesson 3, Exercise 1/reviews_Patio_Lawn_and_Garden_5.json.zip'}

## Lambda Handler

In [2]:
# This cell will write the function to your local machine. Note the name of the file and the name of the function. 
# Compare this to the 'Handler' parameter. 
%%writefile lambda_function.py

import json
from HelloBlazePreprocessLambda import preprocess

def lambda_handler(event, context):
    preprocess(event['s3-dataset-uri'])
    return {
        'statusCode': 200,
        'body': json.dumps("PREPROCESS SUCCESSFULLY!")
    }

# Create Lambda function

In [3]:
import boto3
from sagemaker import get_execution_role
from zipfile import ZipFile

# Your role will need full access to Lambda. The easiest way to add this is through the IAM console.
role = get_execution_role()
client = boto3.client('lambda')

with ZipFile('code.zip', 'w') as f:
    f.write('HelloBlazePreprocessLambda.py')
    f.write('lambda_function.py')
    
# If submitting as a ZipFile, you need to insert raw data.    
with open('code.zip', 'rb') as f:
    b_code = f.read()
    
# You will need to submit an 'execution role' to Lambda. The easiest way to add this is through the IAM console. 
# You'll need the full ARN (not just the name).

response = client.create_function(
    FunctionName='SDKPreprocessLambda',
    Runtime='python3.9',
    Handler='lambda_function.lambda_handler',
    Code={
        'ZipFile': b_code,
    },
    Description='A lambda function made through SDK.',
    Timeout=90,  # Timeout 1 min 30 sec
    MemorySize=1024,
    Publish=True,
    PackageType='Zip',
    Role='arn:aws:iam::988060366449:role/service-role/test-role-ewvcwz8d'
)